In [3]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pymysql
import pandas as pd

# Load CSV
df = pd.read_csv("Final_Choco_Crunch1.csv")

# Split into DataFrames for each table
df_product_info = df[['product_code', 'product_name', 'brand']]
df_nutrient_info = df[
    [
        'product_code', 'energy_kcal_value', 'energy_kj_value', 'carbohydrates_value',
        'sugars_value', 'fat_value', 'saturated_fat_value', 'proteins_value', 'fiber_value',
        'salt_value', 'sodium_value', 'fruits_vegetables_nuts_estimate_from_ingredients_100g',
        'nutrition_score_fr', 'nova_group'
    ]
]
df_derived_metrics = df[
    [
        'product_code', 'sugar_to_carb_ratio', 'calorie_category',
        'sugar_category', 'is_ultra_processed'
    ]
]

# Replace NaN with None to avoid MySQL error
df_product_info = df_product_info.where(pd.notnull(df_product_info), None)
df_nutrient_info = df_nutrient_info.where(pd.notnull(df_nutrient_info), None)
df_derived_metrics = df_derived_metrics.where(pd.notnull(df_derived_metrics), None)

# Connect to MySQL database
conn = pymysql.connect(
    host="localhost",
    port=3306,
    user="root",       
    password="",       
    database="nutrition" 
)
cursor = conn.cursor()

# Create product_info table
cursor.execute("""
CREATE TABLE IF NOT EXISTS product_info (
    product_code VARCHAR(50) PRIMARY KEY,
    product_name TEXT,
    brand TEXT
)
""")

# Create nutrient_info table
cursor.execute("""
CREATE TABLE IF NOT EXISTS nutrient_info (
    product_code VARCHAR(50),
    energy_kcal_value FLOAT,
    energy_kj_value FLOAT,
    carbohydrates_value FLOAT,
    sugars_value FLOAT,
    fat_value FLOAT,
    saturated_fat_value FLOAT,
    proteins_value FLOAT,
    fiber_value FLOAT,
    salt_value FLOAT,
    sodium_value FLOAT,
    fruits_vegetables_nuts_estimate_from_ingredients_100g FLOAT,
    nutrition_score_fr INTEGER,
    nova_group INTEGER,
    FOREIGN KEY product_code
)
""")

# Create derived_metrics table
cursor.execute("""
CREATE TABLE IF NOT EXISTS derived_metrics (
    product_code VARCHAR(50),
    sugar_to_carb_ratio FLOAT,
    calorie_category TEXT,
    sugar_category TEXT,
    is_ultra_processed TEXT,
    FOREIGN KEY product_code 
)
""")

conn.commit()  # Save the table structures

# Insert data into product_info
for idx, row in df_product_info.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO product_info (product_code, product_name, brand)
        VALUES (%s, %s, %s)
    """, (
        row['product_code'],
        row['product_name'],
        row['brand']
    ))

# Insert data into nutrient_info
for idx, row in df_nutrient_info.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO nutrient_info (
            product_code, energy_kcal_value, energy_kj_value, carbohydrates_value, sugars_value,
            fat_value, saturated_fat_value, proteins_value, fiber_value, salt_value, sodium_value,
            fruits_vegetables_nuts_estimate_from_ingredients_100g, nutrition_score_fr, nova_group
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['product_code'], row['energy_kcal_value'], row['energy_kj_value'],
        row['carbohydrates_value'], row['sugars_value'], row['fat_value'],
        row['saturated_fat_value'], row['proteins_value'], row['fiber_value'],
        row['salt_value'], row['sodium_value'],
        row['fruits_vegetables_nuts_estimate_from_ingredients_100g'],
        row['nutrition_score_fr'], row['nova_group']
    ))

# Insert data into derived_metrics
for idx, row in df_derived_metrics.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO derived_metrics (
            product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed
        ) VALUES (%s, %s, %s, %s, %s)
    """, (
        row['product_code'],
        row['sugar_to_carb_ratio'],
        row['calorie_category'],
        row['sugar_category'],
        row['is_ultra_processed']
    ))

conn.commit()  # Save all inserts
cursor.close()
conn.close()


ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near ')' at line 17")

In [16]:
import mysql.connector
import pandas as pd

# Load CSV
df = pd.read_csv("Final_Choco_Crunch1.csv")

# Split into DataFrames for each table
df_product_info = df[['product_code', 'product_name', 'brand']]
df_nutrient_info = df[
    [
        'product_code', 'energy_kcal_value', 'energy_kj_value', 'carbohydrates_value',
        'sugars_value', 'fat_value', 'saturated_fat_value', 'proteins_value', 'fiber_value',
        'salt_value', 'sodium_value', 'fruits_vegetables_nuts_estimate_from_ingredients_100g',
        'nutrition_score_fr', 'nova_group'
    ]
]
df_derived_metrics = df[
    [
        'product_code', 'sugar_to_carb_ratio', 'calorie_category',
        'sugar_category', 'is_ultra_processed'
    ]
]

# Replace NaN with None for MySQL NULL compatibility
df_product_info = df_product_info.where(pd.notnull(df_product_info), None)
df_nutrient_info = df_nutrient_info.where(pd.notnull(df_nutrient_info), None)
df_derived_metrics = df_derived_metrics.where(pd.notnull(df_derived_metrics), None)

# Connect to MySQL database
conn = pymysql.connect(
    host="localhost",
    port=3306,
    user="root",       
    password="",       
    database="nutrition" 
)
cursor = conn.cursor()

# Create product_info table
cursor.execute("""
CREATE TABLE IF NOT EXISTS product_info (
    product_code TEXT PRIMARY KEY,
    product_name TEXT,
    brand TEXT
)
""")

# Create nutrient_info table
cursor.execute("""
CREATE TABLE IF NOT EXISTS nutrient_info (
    product_code TEXT,
    energy_kcal_value FLOAT,
    energy_kj_value FLOAT,
    carbohydrates_value FLOAT,
    sugars_value FLOAT,
    fat_value FLOAT,
    saturated_fat_value FLOAT,
    proteins_value FLOAT,
    fiber_value FLOAT,
    salt_value FLOAT,
    sodium_value FLOAT,
    fruits_vegetables_nuts_estimate_from_ingredients_100g FLOAT,
    nutrition_score_fr INTEGER,
    nova_group INTEGER,
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
""")

# Create derived_metrics table
cursor.execute("""
CREATE TABLE IF NOT EXISTS derived_metrics (
    product_code TEXT,
    sugar_to_carb_ratio FLOAT,
    calorie_category TEXT,
    sugar_category TEXT,
    is_ultra_processed TEXT,
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
""")

conn.commit()  # Save the table structures

# Bulk insert product_info
cursor.executemany("""
    INSERT IGNORE INTO product_info (product_code, product_name, brand)
    VALUES (%s, %s, %s)
""", df_product_info.values.tolist())

# Bulk insert nutrient_info
cursor.executemany("""
    INSERT IGNORE INTO nutrient_info (
        product_code, energy_kcal_value, energy_kj_value, carbohydrates_value, sugars_value,
        fat_value, saturated_fat_value, proteins_value, fiber_value, salt_value, sodium_value,
        fruits_vegetables_nuts_estimate_from_ingredients_100g, nutrition_score_fr, nova_group
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", df_nutrient_info.values.tolist())

# Bulk insert derived_metrics
cursor.executemany("""
    INSERT IGNORE INTO derived_metrics (
        product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed
    ) VALUES (%s, %s, %s, %s, %s)
""", df_derived_metrics.values.tolist())

conn.commit()
cursor.close()
conn.close()


OperationalError: (1170, "BLOB/TEXT column 'product_code' used in key specification without a key length")